In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [1]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np
# library to clean data 
import re  
# Natural Language Tool Kit 
import nltk  
nltk.download('stopwords')  
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
# to remove stopword 
from nltk.corpus import stopwords  
# for Stemming propose  
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction import DictVectorizer

from collections import Counter, OrderedDict
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import models

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

In [0]:
def textCleaning(line):
    # tokenize text and remove puncutation
    line = [word.strip(string.punctuation) for word in line.split(" ")]
    # remove words that contain numbers
    line = [word for word in line if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    line = [x for x in line if x not in stop]
    # remove empty tokens
    line = [t for t in line if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(line)
    # lemmatize text
    line = [WordNetLemmatizer().lemmatize(t[0], get_postag(t[1])) for t in pos_tags]
    # remove words with only one letter
    line = [t for t in line if len(t) > 1]
    # join all
    line = " ".join(line)
    # lower text
    line = line.lower()

    return(line)

In [0]:
def get_postag(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

This is where you will implement two functions to featurize the data:

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  #Your code here
  genres = []
  matrix=[] 
  check = []
  texts = []
  stop_words=set(stopwords.words("english"))

  for i in range(26): 
    matrix.append(corpus[i]['text'])
    genres.append(corpus[i]['genre'])

  for i in range(len(matrix)): 
    matrix[i] = textCleaning(matrix[i])

  cv = CountVectorizer(ngram_range = (1,1),binary = True)
  texts = cv.fit_transform(matrix)
  vocab = (cv.get_feature_names())

  return texts,genres,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  #Your code here
  genres = []
  matrix=[] 
  check = []
  texts = []
  stop_words=set(stopwords.words("english"))

  for i in range(26): 
    matrix.append(corpus[i]['text'])
    genres.append(corpus[i]['genre'])

  for i in range(len(matrix)): 
    matrix[i] = textCleaning(matrix[i])

  cv = CountVectorizer(ngram_range = (1,3), binary = True, min_df = 0.3)
  texts = cv.fit_transform(matrix)
  vocab = (cv.get_feature_names())

  return texts,genres,vocab

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]


  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [0]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [185]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['duke', 'jest', 'apparel', 'gossip', 'reasonable', 'shallow', 'tarry', 'velvet', 'credit', 'pregnant', 'preform', 'aaron', 'prefix', 'pregnantly', 'prefers', 'preferr', 'prefixed', 'prejudicates', 'prenominate', 'premise']
The most informative terms for neg are: ['slay', 'closet', 'bleed', 'slain', 'craves', 'pledge', 'mutiny', 'preeminence', 'prefer', 'preferment', 'preferr', 'prefers', 'prefix', 'prefixed', 'preform', 'pregnant', 'aaron', 'preeches', 'premeditate', 'premise']
----------L2 Norm-----------
The model's average accuracy is 0.846154
The most informative terms for pos are: ['duke', 'jest', 'apparel', 'shallow', 'garment', 'commodity', 'reasonable', 'mutton', 'board', 'marvellous', 'velvet', 'impossible', 'signior', 'clock', 'credit', 'imagination', 'muddy', 'coat', 'colour', 'road']
The most informative terms for neg are: ['slay', 'bleed', 'camp', 'slain', 'eagle'

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [207]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.692308
The most informative terms for pos are: ['duke', 'upon mine', 'jest', 'well exit', 'never hear', 'shallow', 'gossip', 'good sooth', 'read letter', 'rather', 'raw', 'reach', 'rate', 'rat', 'read', 'raven', 'ravish', 'readiness', 'ready', 'reap']
The most informative terms for neg are: ['slay', 'closet', 'mutiny', 'camp', 'bleed', 'stroke', 'craves', 'pledge', 'receiv', 'rat', 'recount', 'rate', 'rather', 'raven', 'ravish', 'raw', 'reach', 'read', 'read letter', 'readiness']
----------L2 Norm-----------
The model's average accuracy is 0.923077
The most informative terms for pos are: ['upon mine', 'duke', 'eye hath', 'upon mine honour', 'apparel', 'well exit', 'jest', 'pray thee', 'shallow', 'garment', 'commodity', 'never hear', 'need fear', 'thee tell', 'credit', 'impossible', 'board', 'good sooth', 'marvellous', 'muddy']
The most informative terms for neg are: ['slay', 'slain', 'bleed', 'camp', 'closet', 'fierce', 'pl

In [100]:
print(corpus[0])

{'genre': 'comedy', 'id': 'allsWellThatEndsWell', 'text': 'alls well that ends well by william shakespeare dramatis personae king of france the duke of florence bertram count of rousillon lafeu an old lord parolles a follower of bertram two french lords serving with bertram steward servant to the countess of rousillon lavache a clown and servant to the countess of rousillon a page servant to the countess of rousillon countess of rousillon mother to bertram helena a gentlewoman protected by the countess a widow of florence diana daughter to the widow violenta neighbour and friend to the widow mariana neighbour and friend to the widow lords officers soldiers etc french and florentine scene rousillon paris florence marseilles act i scene 1 rousillon the count s palace enter bertram the countess of rousillon helena and lafeu all in black countess in delivering my son from me i bury a second husband bertram and i in going madam weep o er my father s death anew but i must attend his majesty 

In [0]:

  tragedy = ['Titus' , 'Andronicus', 'Romeo' , 'Juliet', 'Lear', 'Hamlet', 'Othello', 'Julius' , 'Caesar', 'Macbeth', 'Antony' , 'Cleopatra', 'Coriolanus' , 'Timon' ]
  comedy = ['Comedy' , 'Taming' , 'Shrew', 'Verona', 'Love' , 'Tempest', 'Winter', 'Cymbeline', 'Pericles', 'Measure', 'Troilus', 'Cressida' , 'Twelfth' , 'Like' , 'Merchant' , 'Venice', 'Midsummer' , 'Dream', 'Kinsmen']

  for i in range(len(tragedy)):
    tragedy[i] = clean_text(tragedy[i])
  for i in range(len(comedy)):
    comedy[i] = clean_text(comedy[i])

  st = set(tragedy)
  index = ([i for i, e in enumerate(vocab) if e in st])
  for i in range(26):
    for j in range(len(index)):
      texts[i,int(index[j])] = texts[i,index[j]]*(100)

  st = set(comedy)
  index = ([i for i, e in enumerate(vocab) if e in st])
  for i in range(26):
    for j in range(len(index)):
      texts[i,int(index[j])] = texts[i,index[j]]*(-100)
